In [3]:
pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 14.2 MB/s eta 0:00:00


In [4]:
pip install langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 5.3 MB/s eta 0:00:00


In [5]:
pip install transformers

In [6]:
pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 4.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [7]:
pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 45.9 MB/s eta 0:00:00


In [8]:
import requests
from bs4 import BeautifulSoup as bs
import re
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

In [ ]:
def text_from_website(url):
    try:

      response = requests.get(url)
      response.raise_for_status()  # Raise an exception for bad requests
      soup = bs(response.content, 'html.parser')

      # Removing js and css code
      for script in soup(["script", "style"]):
          script.extract()

      # Extract text content
      text = soup.get_text()

      # Cleaning the text (remove extra whitespaces, etc.)
      text_cleaning = text.strip().replace('\n', ' ')
      cleaned_text = re.sub(r"(\w)-\n(\w)", r"\1\2", text_cleaning)

      return text
    except Exception as e:
        return f"Error: {e}"

In [ ]:
url=''
website_text=text_from_website(url)

In [ ]:
def chunk_text(text):
  '''splitting the corpus into small chunks because LLM have limited context window.
    Splitting text into chunks ensures each chunk fits within this window for better understanding and processing.
    Here we are using RecursiveCharacterTextSplitter from langchain
    '''
  text_splitter = RecursiveCharacterTextSplitter(
            separators=['\n\n', '\n', '.', ','],
            chunk_size=1000,
            chunk_overlap=200,
            )
  texts = text_splitter.split_text(text)
  return texts

texts=chunk_text(website_text)

In [ ]:
def embedding(texts):
  '''
    Then perform the vectorization on those chunks and convert into embedding and stored in vectorstore
    here im using huggingfaceebmbedding with the model which i used to perform retrieve data
    and used faiss vectorstore to store the vectors
    FAISS demonstrates exceptional proficiency in handling high-dimensional data with remarkable speed and efficiency.
    '''


  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
  vector_store = FAISS.from_texts(texts, embeddings)
  return vector_store

vector_store=embedding(texts)

In [ ]:
from langchain.llms import HuggingFaceHub

llm=HuggingFaceHub(repo_id="HuggingFaceH4/zephyr-7b-beta",huggingfacehub_api_token='huggingface api key')

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
You are an AI assistant integrated into a WordPress website, powered by a Retrieval-Augmented Generation (RAG) model. Your task is to retrieve relevant information from a large knowledge base and use that information to generate a final answer.

When given a question, follow these steps:

1. Analyze the Question: Identify the key information needed to answer the question.
2. Retrieve Information: Retrieve relevant information from the knowledge base that could help answer the question.
3. Chain of Thought Reasoning: Walk through the reasoning process step-by-step to arrive at a final answer. This includes:
    - Stating the initial question
    - Listing the relevant information retrieved from the knowledge base
    - Explaining how the retrieved information helps answer the question
    - Connecting the dots and drawing logical conclusions
    - Stating the final answer


Example Chain of Thought:

Question: How do I install a plugin on my WordPress site?

Retrieved Information:
- To install a plugin, you need to access the WordPress dashboard.
- Navigate to the "Plugins" section and click "Add New".
- You can search for plugins in the search bar or upload a plugin file from your computer.
- After finding the desired plugin, click "Install Now" and then "Activate" to enable the plugin.

Chain of Thought:
The question asks how to install a plugin on a WordPress site. From the retrieved information, we learn that you need to access the WordPress dashboard and navigate to the "Plugins" section. Within this section, you can either search for plugins or upload one from your computer. Once you find the desired plugin, you should click "Install Now" and then "Activate" to enable it. By following these steps, you can successfully install and activate a plugin on your WordPress site.

Final Answer: To install a plugin on your WordPress site, go to the WordPress dashboard, navigate to "Plugins" > "Add New", search for the plugin or upload a plugin file, click "Install Now", and then "Activate" to enable the plugin.

By walking through the chain of thought, your responses will be more transparent, allowing users to understand your reasoning process. Please use this approach when answering questions.


Answer the following question based only on the provided context.

<context>
{context}
</context>
Question: {input}""")

In [ ]:
## Chain Introduction
## Create Stuff Docment Chain

from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain=create_stuff_documents_chain(llm,prompt)

In [ ]:
"""
Retrievers: A retriever is an interface that returns documents given
 an unstructured query. It is more general than a vector store.
 A retriever does not need to be able to store documents, only to
 return (or retrieve) them. Vector stores can be used as the backbone
 of a retriever, but there are other types of retrievers as well.
"""

retriever=vector_store.as_retriever()
retriever

In [ ]:
"""
Retrieval chain:This chain takes in a user inquiry, which is then
passed to the retriever to fetch relevant documents. Those documents
(and original inputs) are then passed to an LLM to generate a response
"""
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [ ]:
query="what are the Challenges and Ethical Considerations"
response=retrieval_chain.invoke({"input":query})
ans=response['answer']
ans

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)

In [ ]:
while True:
  query = input("Ask a question about the website ('END' to exit): ")
  if query.upper() == "END":  # Check for "END/end"
      break
  response = retrieval_chain.invoke({"input": query}) # retrieve the answer
  answer = response["answer"]


  # to remove the prompt from the output

  start_marker = "the following question based only on the provided context"

  # Find the index of the start marker
  start_index = answer.find(start_marker)

  if start_index != -1:
      # Extract the generated output after the marker
      generated_output = answer[start_index + len(start_marker):].strip()

      # Preserve the formatting by splitting lines
      formatted_output = "\n".join(line.strip() for line in generated_output.splitlines() if line.strip())

      # Print or use the formatted output
      print(formatted_output)
  else:
      print("Start marker not found in the generated text.")

  memory.save_context({"input": query}, {"output": formatted_output})
  print('---'*10)
  print(formatted_output)